# Point-Source Photometry

If it's one thing an observational astronomer can do, it's measuring the flux of something. The easiest of these somethings are stars. We will see why that is.

When we talk about **flux** in the context of this notebook, what we mean is the number of photons recorded in an image like this:

![](https://apod.nasa.gov/apod/image/0202/ngc6397_hst.jpg)

Fluxes are often converted to magnitudes, a logarithmic measure. This goes back to the practice of ancient Arabs, who classified the brightest stars with number 1, and the faintest one they could see with number 6. The most common convention today is the so-called AB magnitude:

$$
m_\text{AB} = -2.5 \log_{10} f - 48.60
$$

where $f$ is the *spectral* flux in units of $erg/s/cm^2/Hz$, so it's the energy received by the instrument per time, collecting area and frequency.

This all boils down to counting photons.

## Exercise 1:

FITS is the ubiquitous file format in astronomy. Open the file `data/point-source.fits` with [astropy](https://docs.astropy.org/en/stable/io/fits/). Plot the image.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np


We can see a star with an apparently Gaussian shape, plus a fair bit of noise. 

Btw, what determines the shape of a star?

## Exercise 2:

Use astropy's `Gaussian2D.evaluate` [(docs)](https://docs.astropy.org/en/stable/api/astropy.modeling.functional_models.Gaussian2D.html) to create in image of a circular Gaussian model that looks similar to the data above. How many effective parameters does this model have?

Also, `numpy.meshgrid` is quite useful!

## Exercise 3:

Assume that your guess for the position of the stars is correct, or that somebody told you the exact position (if the star is bright enough, Gaia can tell you very precisely its position on the sky). 
Finding the flux of the star is then a linear problem:

$$
I = f \cdot \mathrm{Gaussian} + s \cdot\mathrm{const} + \mathrm{noise}
$$

Flatten the image and write down the design matrix, including a constant term. 

Like all counting experiments, the noise in this image follows a **Poisson distribution**. The nice thing about the Poisson distribution is that it quickly looks like the Gaussian when the counts are high enough. In this case, the noise is very well approximated by

$$
\sigma_i^2 = I_i
$$

where $i$ is the pixel index, and $I_i$ the intensity value of the pixel. Construct the covariance matrix $\Sigma$ for the flattened image, and determine the parameters $f$ and $s$ by solving the ordinary least-squares problem. Don't forget the covariance matrix! What's the meaning of the constant term?

## Exercise 4:

Assume now that you don't know the position, or at least not well enough. Use `scipy.optimimze.minimize` to fit 5 parameters: $f$ and $s$ (as above) as well as the position $x_m$, $y_m$, and the width (i.e. standard deviation) of the Gaussian.

The function you want to minimize to find the "best-fit" parameters is the one that maximizes the likelihood. Since we're dealing with (assumed) Gaussian errors, this is the usual quadratic error form

$$
\chi^2(f, s, x_m, y_m, w;\ \lbrace I_i \rbrace) = \sum_i^N \frac{\left(I_i - \mathrm{Gaussian}(x_i, y_i; f, x_m, y_m, w) - s\right)^2}{\sigma_i^2},
$$

where $x_i$ and $y_i$ are the position of pixel $i$.

> Minimizing $\chi^2$ means altering the *parameters* at fixed data!

Note that `minimize` assumes that the function takes the entire parameter vector as **the first argument**!

So, a good function signature for this task is:

```python
def chi2(params, img):
    f, s, xm, ym, w = params
    ...

x0 = (160, 100, 16, 16, 3)
result = so.minimize(chi2, x0, args=(img, ))
```

This uses a convenient separation of optimization parameters from other arguments `chi2` needs for its computation. It also frees you to define the parameter vector `params` any way you want as long as its consistent everywhere.

Also, for `x0`, start with your best-guess for the positions as well as the results from Exercise 3.

How would you determine if the fit is actually good? There are multiple ways, but often the most intuitive is to look at the residuals, i.e. the difference between data and model.

Make a plot of your best-fit residuals.

It *should* look like your error model, which in our cause is a non-uniform Gaussian with higher variances in regions with higher pixel intensity.